## Read .ghg metadata

In [1]:
import os
import time
import pandas as pd
from Parse_GHG import read_GHG
from ipywidgets import FloatProgress
from IPython.display import display

# Site = 'BB2'

# Values we expect to change
dynamicMetaData_Headers = ['altitude','canopy_height']

# Values we want to fix
staticMetaData_FloatHeaders = ['altitude', 'latitude', 'longitude']
staticMetaData_TextHeaders = ['site_name']

T1 = time.time()
print('Processing ghg files ')
f = FloatProgress(min=0, max=1) 
display(f) 
# Walk through the directory to find all "raw" folders

N_recs = 4#8*31
def find_new_GHG(Site,make_new=False):

    i = 0
    Dpath = 'C:\\highfreq\\'
    raw_dir = Dpath+Site+'\\raw\\'
    meta_dir = Dpath+Site+'\\metadata\\'
    meta_file = meta_dir+'GHGMetaData.csv'

    if os.path.isfile(meta_file) and make_new is False:
        Records = pd.read_csv(meta_file,parse_dates=['TimeStamp'])
        FileNames = Records['filename'].tolist()
    else:
        Records = pd.DataFrame()
        FileNames = []
        if not os.path.exists(meta_dir):
            os.mkdir(meta_dir)
    

    for (root, dirs, files) in sorted(os.walk(raw_dir)):
        # if root.__contains__('raw') :
        for file in files:
            name, tag = file.split('.')
            # .ghg files are located at the end of each directory tree
            # Avoids reading any that might be misplaced elsewhere
            if tag == 'ghg' and len(dirs)==0 and file not in FileNames:  
                
                if i <= 48*5:
                    GHG_out = read_GHG(root,name)
                    Records = pd.concat(
                        [Records,GHG_out.Summary],
                    axis=0,
                    ignore_index=True
                    )
                    i += 1
            f.value = i/len(files)
    return(Records,meta_dir,i)

Records,meta_dir,i = find_new_GHG('BB')

Records.to_csv(meta_dir+'GHGMetaData.csv',index=False)

elapsed = time.time()-T1
print('Processing time ', elapsed) 
print('To inspect ', i, ' .ghg files')
print('Seconds/file ', elapsed/(i+1)) 

staticMetaData = Records[staticMetaData_FloatHeaders].astype(float).mean()
staticMetaData = pd.concat([Records[staticMetaData_TextHeaders].astype(str).mode().T[0],staticMetaData])
print(staticMetaData)

dynamicMetaData = Records.set_index('TimeStamp')[dynamicMetaData_Headers]
dynamicMetaData['date']=dynamicMetaData.index.date
dynamicMetaData['time']=dynamicMetaData.index.time
dynamicMetaData.to_csv(meta_dir+'dynamicMetaData.csv',index=False)
dynamicMetaData

# Processing time  65.87000775337219
# To inspect  241  .ghg files

Records['altitude'].astype(float).describe()


Processing ghg files 


FloatProgress(value=0.0, max=1.0)

                  index       0      1
0      Diagnostic Value  8189.0    NaN
1    Diagnostic Value 2     1.0    NaN
2  CH4 Diagnostic Value   143.0  207.0


ValueError: Length mismatch: Expected axis has 3 elements, new values have 2 elements